## 附录 10.2：工具使用

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格，以加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:
%pip install anthropic

# 导入 Python 内置的正则表达式库
import re
import anthropic

# 从 IPython 存储中检索 API_KEY 变量
%store -r API_KEY

client = anthropic.Anthropic(api_key=API_KEY)

# 重写以调用 Claude 3 Sonnet 模型，该模型通常更擅长工具使用，并包含停止序列
def get_completion(messages, system_prompt="", prefill="",stop_sequences=None):
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages,
        stop_sequences=stop_sequences
    )
    return message.content[0].text

---

## 课程

虽然“工具使用”（又称“函数调用”）乍听起来概念复杂，但它实际上相当简单！你已经掌握了实现工具使用所需的所有技能，它本质上只是“替换”和“提示链”的组合。

在之前的替换练习中，我们将文本替换到提示中。而在工具使用中，我们将工具或函数的结果替换到提示中。Claude 无法真正地“调用”或“访问”工具和函数。相反，我们让 Claude：
1. 输出它想要调用的工具名称和参数
2. 暂停任何进一步的响应生成，同时调用该工具
3. 然后我们用附加了工具结果的新提示重新向其提问

函数调用之所以有用，是因为它扩展了 Claude 的能力，并使 Claude 能够处理更复杂、多步骤的任务。
你可以赋予 Claude 的一些函数示例包括：
- 计算器
- 单词计数器
- SQL 数据库查询和数据检索
- 天气 API

你可以通过结合以下两个要素来让 Claude 实现工具使用：

1.  一个系统提示：我们在其中向 Claude 解释工具使用的概念，并提供它可访问工具的详细描述列表。
2.  用于编排和执行 Claude 工具使用请求的控制逻辑。

### 工具使用路线图

*本课程教授我们当前的工具使用格式。然而，我们将在不久的将来更新和改进工具使用功能，包括：*
*   *更精简的函数定义和调用格式*
*   *更强大的错误处理和边缘情况覆盖*
*   *与我们 API 其余部分的更紧密集成*
*   *更好的可靠性和性能，特别是对于更复杂的工具使用任务*

### 示例

为了在 Claude 中启用工具使用，我们从系统提示开始。在这个特殊的工具使用系统提示中，我们告诉 Claude：
*   工具使用的基本前提及其含义
*   Claude 如何调用和使用它被赋予的工具
*   在此特定场景中，它可访问工具的详细列表

这是系统提示的第一部分，向 Claude 解释工具使用。系统提示的这一部分可以推广到所有提示 Claude 进行工具使用的实例中。我们赋予 Claude 的工具调用结构（`<function_calls> [...] </function_calls>`）是 Claude 经过专门训练使用的结构，因此我们建议您坚持使用此结构。

In [ ]:
system_prompt_tools_general_explanation = """您可以使用一组函数来回答用户的问题。这包括访问一个沙盒计算环境。您目前无法检查文件或与外部资源交互，除非通过调用以下函数。

您可以通过在回复用户时写入如下所示的 "<function_calls>" 块来调用一个或多个函数：
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<invoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

字符串和标量参数应按原样指定，而列表和对象应使用 JSON 格式。请注意，字符串值中的空格不会被去除。输出不期望是有效的 XML，而是通过正则表达式解析。

输出和/或任何错误将出现在随后的 "<function_results>" 块中，并作为您回复用户的一部分保留在那里。
然后您可以继续撰写回复用户的其余部分，响应任何错误，或根据需要进行进一步的函数调用。
如果您的函数调用后没有出现 "<function_results>"，那么它们很可能格式不正确，未被识别为调用。"""

这是系统提示的第二部分，它定义了 Claude 在此特定情境下可以访问的精确工具。在此示例中，我们将为 Claude 提供一个计算器工具，该工具接受三个参数：两个操作数和一个运算符。

然后我们将系统提示的两个部分组合起来。

In [ ]:
system_prompt_tools_specific_tools = """这里是 JSONSchema 格式的可用函数：
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
用于执行基本算术运算的计算器函数。
支持加法、减法、乘法
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>第一个操作数（运算符之前）</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>第二个操作数（运算符之后）</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>要执行的操作。必须是 +、-、* 或 / 之一</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

现在我们可以给 Claude 一个需要使用 `calculator` 工具的问题。我们将使用 `<function_calls\>` 作为 `stop_sequences` 来检测 Claude 是否以及何时调用函数。

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "将 1,984,135 乘以 9,343,116"
}

stop_sequences = ["</function_calls>"]

# 获取 Claude 的响应
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

现在，我们可以从 Claude 的函数调用中提取参数，并代表 Claude 实际运行该函数。

首先，我们将定义函数的代码。

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "错误：不支持的操作。"

然后，我们将从 Claude 的函数调用响应中提取参数。如果所有参数都存在，我们就运行计算器工具。

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- 结果 ----------------")
    print(f"{result:,}")

现在我们有了一个结果，我们必须正确地格式化这个结果，以便当我们将其传回给Claude时，Claude能够理解该结果与哪个工具相关。Claude被训练识别一种固定的格式：
```
<function_results>
<result>
<tool_name>{工具名称}</tool_name>
<stdout>
{工具结果}
</stdout>
</result>
</function_results>
```

运行下面的单元格，将上述工具结果格式化为这种结构。

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

现在我们只需将这个结果附加到与之前相同的消息链中，然后将其发回给Claude，这样就大功告成了！

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# 构建完整的对话
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]

# 打印Claude的响应
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- 最终结果 -------------")
print(final_response)

恭喜你，成功地运行了一个完整的工具使用链端到端！

现在，如果我们给Claude一个完全不需要使用给定工具的问题，会怎么样呢？

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "告诉我法国的首都是哪里。"
}

stop_sequences = ["</function_calls>"]

# 获取Claude的响应
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

成功了！正如你所看到的，当不需要调用函数时，Claude知道不进行调用。

如果你想在不更改上方任何内容的情况下，尝试本课程的提示，请一直滚动到课程笔记本的底部，访问[**示例演练场**](#example-playground)。

---

## 练习
- [练习 10.2.1 - SQL](#exercise-1021---SQL)

### 练习 10.2.1 - SQL
在这个练习中，你将编写一个用于查询和写入世界上最小“数据库”的工具使用提示。以下是已初始化的数据库，它实际上只是一个字典。

In [ ]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

这是用于向数据库写入数据和从数据库读取数据的函数代码。

In [ ]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

要解决这个练习，首先定义一个系统提示，就像上面提到的 `system_prompt_tools_specific_tools` 那样。请务必包含每个工具的名称和描述，以及每个函数的每个参数的名称、类型和描述。我们已经在下面为你提供了一些起始的脚手架代码。

In [ ]:
system_prompt_tools_specific_tools_sql = """
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

准备好后，你可以在下面的示例中尝试你的工具定义系统提示。只需运行下面的单元格即可！

In [ ]:
examples = [
    "向数据库添加一个名为Deborah的用户。",
    "向数据库添加一个名为Thingo的产品。",
    "告诉我用户2的名称。",
    "告诉我产品3的名称。"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # 获取并打印Claude的响应
    function_calling_response = get_completion([message], system_prompt=system_prompt, stop_sequences=stop_sequences)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

如果操作正确，函数调用消息应该能正确调用 `add_user`、`add_product`、`get_user` 和 `get_product` 函数。

作为额外加分项，请添加一些代码单元格并编写参数解析代码。然后使用Claude给出的参数调用这些函数，以查看调用后“数据库”的状态。

❓ 如果你想查看一个可能的解决方案，请运行下面的单元格！

In [ ]:
from hints import exercise_10_2_1_solution; print(exercise_10_2_1_solution)

### 恭喜！

恭喜你学会了工具使用和函数调用！如果你想了解更多关于搜索和RAG的知识，请前往最后一个附录部分。

---

# Appendix 10.2: Tool Use

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
%pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY variable from the IPython store
%store -r API_KEY

client = anthropic.Anthropic(api_key=API_KEY)

# Rewrittten to call Claude 3 Sonnet, which is generally better at tool use, and include stop_sequences
def get_completion(messages, system_prompt="", prefill="",stop_sequences=None):
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages,
        stop_sequences=stop_sequences
    )
    return message.content[0].text

---

## Lesson

While it might seem conceptually complex at first, tool use, a.k.a. function calling, is actually quite simple! You already know all the skills necessary to implement tool use, which is really just a combination of substitution and prompt chaining.

In previous substitution exercises, we substituted text into prompts. With tool use, we substitute tool or function results into prompts. Claude can't literally call or access tools and functions. Instead, we have Claude:
1. Output the tool name and arguments it wants to call
2. Halt any further response generation while the tool is called
3. Then we reprompt with the appended tool results

Function calling is useful because it expands Claude's capabilities and enables Claude to handle much more complex, multi-step tasks.
Some examples of functions you can give Claude:
- Calculator
- Word counter
- SQL database querying and data retrieval
- Weather API

You can get Claude to do tool use by combining these two elements:

1. A system prompt, in which we give Claude an explanation of the concept of tool use as well as a detailed descriptive list of the tools it has access to
2. The control logic with which to orchestrate and execute Claude's tool use requests

### Tool use roadmap

*This lesson teaches our current tool use format. However, we will be updating and improving tool use functionality in the near future, including:*
* *A more streamlined format for function definitions and calls*
* *More robust error handling and edge case coverage*
* *Tighter integration with the rest of our API*
* *Better reliability and performance, especially for more complex tool use tasks*

### Examples

To enable tool use in Claude, we start with the system prompt. In this special tool use system prompt, wet tell Claude:
* The basic premise of tool use and what it entails
* How Claude can call and use the tools it's been given
* A detailed list of tools it has access to in this specific scenario 

Here's the first part of the system prompt, explaining tool use to Claude. This part of the system prompt is generalizable across all instances of prompting Claude for tool use. The tool calling structure we're giving Claude (`<function_calls> [...] </function_calls>`) is a structure Claude has been specifically trained to use, so we recommend that you stick with this.

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<invoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

Here's the second part of the system prompt, which defines the exact tools Claude has access to in this specific situation. In this example, we will be giving Claude a calculator tool, which takes three parameters: two operands and an operator. 

Then we combine the two parts of the system prompt.

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

Now we can give Claude a question that requires use of the `calculator` tool. We will use `<function_calls\>` in `stop_sequences` to detect if and when Claude calls the function.

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

Now, we can extract out the parameters from Claude's function call and actually run the function on Claude's behalf.

First we'll define the function's code.

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

Then we'll extract the parameters from Claude's function call response. If all the parameters exist, we run the calculator tool.

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

Now that we have a result, we have to properly format that result so that when we pass it back to Claude, Claude understands what tool that result is in relation to. There is a set format for this that Claude has been trained to recognize:
```
<function_results>
<result>
<tool_name>{TOOL_NAME}</tool_name>
<stdout>
{TOOL_RESULT}
</stdout>
</result>
</function_results>
```

Run the cell below to format the above tool result into this structure.

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

Now all we have to do is send this result back to Claude by appending the result to the same message chain as before, and we're good!

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

Congratulations on running an entire tool use chain end to end!

Now what if we give Claude a question that doesn't that doesn't require using the given tool at all?

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

Success! As you can see, Claude knew not to call the function when it wasn't needed.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 10.2.1 - SQL](#exercise-1021---SQL)

### Exercise 10.2.1 - SQL
In this exercise, you'll be writing a tool use prompt for querying and writing to the world's smallest "database". Here's the initialized database, which is really just a dictionary.

In [ ]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

And here is the code for the functions that write to and from the database.

In [ ]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

To solve the exercise, start by defining a system prompt like `system_prompt_tools_specific_tools` above. Make sure to include the name and description of each tool, along with the name and type and description of each parameter for each function. We've given you some starting scaffolding below.

In [ ]:
system_prompt_tools_specific_tools_sql = """
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

When you're ready, you can try out your tool definition system prompt on the examples below. Just run the below cell!

In [ ]:
examples = [
    "Add a user to the database named Deborah.",
    "Add a product to the database named Thingo",
    "Tell me the name of User 2",
    "Tell me the name of Product 3"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # Get & print Claude's response
    function_calling_response = get_completion([message], system_prompt=system_prompt, stop_sequences=stop_sequences)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

If you did it right, the function calling messages should call the `add_user`, `add_product`, `get_user`, and `get_product` functions correctly.

For extra credit, add some code cells and write parameter-parsing code. Then call the functions with the parameters Claude gives you to see the state of the "database" after the call.

❓ If you want to see a possible solution, run the cell below!

In [ ]:
from hints import exercise_10_2_1_solution; print(exercise_10_2_1_solution)

### Congrats!

Congratulations on learning tool use and function calling! Head over to the last appendix section if you would like to learn more about search & RAG.

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Get Claude's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)